# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving requirements.txt to requirements.txt
User uploaded file "requirements.txt" with length 226 bytes


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB

In [4]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

In [5]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [6]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

configuration_chatglm.py:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_chatglm.py:   0%|          | 0.00/55.9k [00:00<?, ?B/s]

...

model-00004-of-00007.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

## 使用微调前 ChatGLM3

In [9]:
input_text = "解释下乾卦是什么？"

In [10]:
response, history = base_model.chat(tokenizer, query=input_text)

In [11]:
print(response)

乾卦是八卦之一，也是八宫图中的第一个卦。乾卦是由两个阳爻夹一个阴爻构成，象征着天、云、雷等阳性的气象现象。乾卦的含义包括力量、刚强、积极、刚正不阿等，同时也象征着一种强烈的进取心和冒险精神。

在八宫图中，乾卦位于北方，与事业、努力、坚定、果敢等有关。它代表着一个积极向上的态度和一种不屈不挠的精神，同时也象征着力量和勇气。在个人的生命中，乾卦的位置也可以反映出一个人在事业和生活中的积极性和刚强精神。

乾卦还可以引申为“天”，表示一个 higher power or divine force that governs the universe. In this sense,乾卦 can be seen as a symbol of hope and inspiration, reminding us to trust in the greater plan and to strive towards our goals with confidence and determination.


In [12]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是八卦中的第68卦，由两个阴爻夹一个阳爻构成。师卦的含义包括教导、指导、领导、启发等，象征着一种引导和启示的作用。

在八宫图中，地水师卦位于西方，与知识、学习、智慧、口才等有关。它代表着一种智慧、聪明和口才优秀的特点，同时也象征着一种善于教导和引导他人的能力。

在个人的生命中，地水师卦的位置也可以反映出一个人在知识、学习、智慧和口才方面的优秀表现。同时，地水师卦还可以引申为“水”，表示柔性和适应性，提醒人们在面对生活和工作的挑战时，要具备灵活变通和适应能力，以便更好地实现自己的目标。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"/content/drive/MyDrive/models/{model_name_or_path}-epoch{epochs}-20240209_084913"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [15]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)

    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B (Epoch=3, automade-dataset) 微调后：\n{ft_response}")
    return base_response, ft_response

In [20]:
base_response, ft_response = compare_chatglm_results(query="解释下临卦是什么？")

问题：解释下临卦是什么？

原始输出：
临卦，是周易卦象中的一种，象征着大吉大利，吉利的卜问。然而在八月可能有凶险之事。其卦象为坤上兑下，堤岸高出大泽，河泽容于大地，意味着君临天下，教化万民，保容万民，德业无疆。这一卦代表着池泽之水灌溉地面万物，相辅相成，生生不息，意味着诸事亨通，上下和合，前途有望。然而，需注意不宜过于急进。

临卦蕴含着好运来临，经营有成，家业正旺，疾病拖延但不致危险的时机。在事业上，正是成功的极好时机，务必抓紧，不可失去良机，但不可就此满足。在经商时，会十分顺利，并有收获，但应谨慎处理人际关系。在决策中，务必加强个人的全面修养，诚信待人，树立良好的个人威望，有远大的眼光，长久的计划，可以保证事业的发达和前程的光明。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 解释下临卦是什么？ 临卦，是周易卦象中的一种，象征着大吉大利，吉利的卜问。然而在八月可能有凶险之事。其卦象为坤上兑下，堤岸高出大泽，河泽容于大地，意味着君临天下，教化万民，保容万民，德业无疆。这一卦代表着池泽之水灌溉地面万物，相辅相成，生生不息，意味着诸事亨通，上下和合，前途有望。然而，需注意不宜过于急进。

临卦蕴含着好运来临，经营有成，家业正旺，疾病拖延但不致危险的时机。在事业上，正是成功的极好时机，务必抓紧，不可失去良机，但不可就此满足。在经商时，会十分顺利，并有收获，但应谨慎处理人际关系。在决策中，务必加强个人的全面修养，诚信待人，树立良好的个人威望，有远大的眼光，长久的计划，可以保证事业的发达和前程的光明。


In [18]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦，又称作观卦，是周易卦象中的一种，象征着大吉大利，吉利的卜问。这一卦象代表水，象征着你正在观察活水从源头流过，应当时机成熟，勇敢地决定时机，否则便会导致时机的消失。

地水师卦的核心哲学是：万物相克相生，时机成熟时应当勇敢地主动进攻，以取得最佳结果。

此卦预示着运势较低，决定采用主动进攻的策略，以提高成功的概率。在事业、财运等方面都应勇敢乐观，积极进取，以免错过时机而导致时机的消失。

对于个人运势，地水师卦预示着运势较好，但在事业、财运方面需要积极进攻，以取得最佳结果。在事业方面，应勇敢地面对困难，积极进取，以免失去时机而导致错失良机。在财运方面，适宜积极进攻，提高财运，同时要谨慎行事，以免遭遇不幸。

总体来说，地水师卦预示着勇敢地面对困难，积极进取，时机成熟时应主动进攻，以取得最佳结果。在失落时应坚定信念，耐尽艰辛，等待时机，一旦出现便应勇敢地进攻，以取得最佳结果。"


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 地水师卦是什么？ 在周易中，卦象为乾卦和坤卦相叠，象征女子贤良、温顺而柔弱，同时也刚毅、刚健而强大。这种卦象被称为“坤德”，表示大地充满生机，女子具有贤良之德。

传统解卦认为，此卦为贤良之女夫，宜嫁比自己年长、德高的人，则家宅受益。对于求婚者，则表示婚姻美满，对于商人则预示着生意顺利，但 also暗示可能面临风险和考验。

此外，此卦在职场也预示着 initial 成功，但 also 需要谨慎行事，以避免损失。对于创业者，表示创业初段阶段顺利，但 also 需要谨慎行事，以避免出现波折。

在事业上，表示 initial 顺利，但也需要谨慎行事，以避免出现波折，初始阶段可能顺利，后续则需要谨慎行事，以避免出现波折，初始阶段顺利，后续则可能出现小的失误或变故，需要小心处理。

总结起来，此卦表示初始顺利，后续需要谨慎行事，以避免出现波折。对于事业初始者，初始顺利，后续需要谨慎行事，以避免出现波折，初始阶段顺利，后续则可能出现小的失误或变故，需要小心处理。

总结起来，此卦表示 initial 顺利，后续需要谨慎行事，以避免出现波折。对于事业初始者，初始顺利，后续需要谨慎行事，以避免出现波折。对于求婚者， initial 顺利，后续也需

In [21]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
{'name': '日在天顶，函谷之风吹过，使人感到熱热的，这正是讼卦的卦象。在这个卦象中，上卦是乾，代表天；下卦是坎，代表田地，中间的杠是阳刚之卦，表示天地之间的相争。因此，这个卦象代表天地之间的争鸣，意味着江湖上的争斗。需要通过卜卦者对卦象的解读，才能得知问题的解决之道。', 'content': '\n讼卦是一个凶卦，但通过合理运用，可以转危为安。在运势中，意味着可能面临困难与挫折，需要小心应对；在事业上，可能会有争夺和竞争，需要谨慎行事；在经商时，则视为凶运，需要小心谨慎，避免冒险。\n\n总的来说，讼卦代表着争斗和困境，需要谨慎小心地处理争端，严格和技术，避免冒险。'}


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 周易中的讼卦是什么?" 讼卦"是《周易》中的一卦，卦象由上卦乾和下卦坎组成。乾象征天，坎象征水。天助助水，使水上升至高远之处。这一卦象暗示着君子 assist the resolution of conflicts，即助长众人的不同特性，以达到和谐 Perfection。因此，讼卦是象征着君子 assist the resolution of conflicts的卦象。

在古代，人们通过解卦来指导 Behavior and Thought。在"讼卦"中，君子应当助长仁爱、助长敬重、辅佐正道，以得到众人的支持。

"讼卦"的卦象为天助水，也是君子 sample of the year，意味着君子 assist the resolution of conflicts，也象征着德高望重。

在古代，解卦为君子德高望重，助长敬重，助长仁爱，辅佐正道，以得到众人的支持。

因此，"讼卦"是象征着君子 assist the resolution of conflicts的卦象，也预示着君子 Assist the resolution of conflicts，得到众人的支持。


In [22]:
base_response, ft_response = compare_chatglm_results(query="如何理解周易中的坤卦")

问题：如何理解周易中的坤卦

原始输出：
坤卦，又称“元”卦，是八卦中的第二卦。它是由两个坤卦叠加而成的，象征大地之宽广，代表着顺从、承载和生育万物的阴性。坤卦中有“元、亨、利牝马之贞”的卦辞，意味着大吉大利，雌马得到吉兆。在旅行时，会先迷失方向，后来才找到主人，西南行会得朋友，东北行则丧失朋友。在安定定居方面也是吉利之兆。

        坤卦的核心哲学是：顺应天道，以厚德载物。此卦象征着包容和柔顺，要以深厚的德行来承担重大责任。因此，它要求人们顺应运势，以静制动，不宜独立谋事，而应通过顺应他人，与人合作而成就大事。在时运、财运、家宅和身体等方面均吉祥。

        传统解卦认为，坤卦象征大地承载万物，以德服众，强调诸事不宜急进，以静制动为宜。在事业、经商、求名、婚恋、决策等方面，都需要以柔克刚，稳健包容，不得急躁冒进，而应埋头苦干，充实自己的内在修养，尊重正道，谋求事业的成功。"


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 如何理解周易中的坤卦 坤卦，又称“元”卦，是八卦中的第二卦。它是由两个坤卦叠加而成的，象征大地之宽广，代表着顺从、承载和生育万物的阴性。坤卦中有“元、亨、利牝马之贞”的卦辞，意味着大吉大利，雌马得到吉兆。在旅行时，会先迷失方向，后来才找到主人，西南行会得朋友，东北行则丧失朋友。在安定定居方面也是吉利之兆。

        坤卦的核心哲学是：顺应天道，以厚德载物。此卦象征着包容和柔顺，要以深厚的德行来承担重大责任。因此，它要求人们顺应运势，以静制动，不宜独立谋事，而应通过顺应他人，与人合作而成就大事。在时运、财运、家宅和身体等方面均吉祥。

        传统解卦认为，坤卦象征大地承载万物，以德服众，强调诸事不宜急进，以静制动为宜。在事业、经商、求名、婚恋、决策等方面，都需要以柔克刚，稳健包容，不得急躁冒进，而应埋头苦干，充实自己的内在修养，尊重正道，谋求事业的成功。"


In [23]:
base_response, ft_response = compare_chatglm_results(query="坤卦的主要哲学思想是什么")

问题：坤卦的主要哲学思想是什么

原始输出：
坤卦，又称“元”卦，是八卦中的第二卦。它是由两个坤卦叠加而成的，象征大地之宽广，代表着顺从、承载和生育万物的阴性。坤卦中有“元、亨、利牝马之贞”的卦辞，意味着大吉大利，雌马得到吉兆。在旅行时，会先迷失方向，后来才找到主人，西南行会得朋友，东北行则丧失朋友。在安定定居方面也是吉利之兆。

        坤卦的核心哲学是：顺应天道，以厚德载物。此卦象征着包容和柔顺，要以深厚的德行来承担重大责任。因此，它要求人们顺应运势，以静制动，不宜独立谋事，而应通过顺应他人，与人合作而成就大事。在时运、财运、家宅和身体等方面均吉祥。

        传统解卦认为，坤卦象征大地承载万物，以德服众，强调诸事不宜急进，以静制动为宜。在事业、经商、求名、婚恋、决策等方面，都需要以柔克刚，稳健包容，不得急躁冒进，而应埋头苦干，充实自己的内在修养，尊重正道，谋求事业的成功。"


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 坤卦的主要哲学思想是什么？ 坤卦，又称“元”卦，是八卦中的第二卦。它是由两个坤卦叠加而成的，象征大地之宽广，代表着顺从、承载和生育万物的阴性。坤卦中有“元、亨、利牝马之贞”的卦辞，意味着大吉大利，雌马得到吉兆。在旅行时，会先迷失方向，后来才找到主人，西南行会得朋友，东北行则丧失朋友。在安定定居方面也是吉利之兆。

        坤卦的核心哲学是：顺应天道，以厚德载物。此卦象征着包容和柔顺，要以深厚的德行来承担重大责任。因此，它要求人们顺应运势，以静制动，不宜独立谋事，而应通过顺应他人，与人合作而成就大事。在时运、财运、家宅和身体等方面均吉祥。

        传统解卦认为，坤卦象征大地承载万物，以德服众，强调诸事不宜急进，以静制动为宜。在事业、经商、求名、婚恋、决策等方面，都需要以柔克刚，稳健包容，不得急躁冒进，而应埋头苦干，充实自己的内在修养，尊重正道，谋求事业的成功。"
